In [97]:
# Import Required Modules
from flask import Flask, render_template
import pandas as pd
import json
import plotly
import plotly.express as px
import pandas_ta as pta
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import date
from datetime import datetime
from datetime import timedelta, date
from prophet import Prophet
import pandas as pd
import numpy as np
import yfinance as yf
import seaborn as sns
import mpld3
import matplotlib.pyplot as plt
import warnings

   
df = pd.read_csv("Resources/btcjoin.csv", parse_dates=['date'])
btc_df = yf.Ticker('BTC-USD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.loc[(btc_df['Date'] > '2022-10-25')]
btc_df['Close']=btc_df['Close'].astype("float")
df['price']=df['price'].str.replace(',','')
df['price']=df['price'].astype("float")
btc_df = btc_df.rename(columns={"Close": "price", "Date":"date"})
btc_df['date'] = btc_df['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
btc_df['date'] = pd.to_datetime(btc_df['date'])

df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = pd.merge(df, btc_df, on=['date', 'price'], how='outer')
df = df.rename(columns={"value": "wallets"})
df = df.drop(columns=['volume','change', 'low', 'high', 'open','Unnamed: 0', "wallets", "address", "mined"])
df['200D'] = df['price'].rolling(200).mean()
df['300D'] = df['price'].rolling(300).mean()
df['50D'] = df['price'].rolling(50).mean()
df['7D'] = df['price'].rolling(7).mean()
# df = df.dropna()
df['meanavge'] = (df['200D'] + df['300D'] + df['50D'] )/3
# df = df.drop(columns=['200D','300D', '50D'])
df['meanvalue'] = df["price"] - df["meanavge"]
df['status'] = df['meanvalue'].apply(lambda x: '1' if x > 0 else '0')
df['status']=df['status'].astype("object")
df['price-meanavge']=df['price'] - df['meanavge']
df['move%'] = (df['price-meanavge']/(df['price'] + df['meanavge']))
bins = [-.43, -.18, 0, .18, .43]
group_names = ["Severely Oversold","Neutral Oversold", "Neutral Overbought","Severely Overbought"]
df["Valuation"] = pd.cut(df["move%"], bins, labels=group_names)

c:\Users\chris\anaconda3\envs\python_data\Lib\site-packages\yfinance\utils.py:775: FutureWarning:

The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.



In [98]:
df['daily_pct_change'] = df['price'].pct_change() * 100
df.tail(10)

,date,price,Open_x,High_x,Low_x,Volume_x,Open_y,High_y,Low_y,Volume_y,...,300D,50D,7D,meanavge,meanvalue,status,price-meanavge,move%,Valuation,daily_pct_change
4951,2024-05-13,62901.449219,61451.218750,63422.660156,60769.839844,2.788918e+10,61451.218750,63422.660156,60769.839844,2.788918e+10,...,43438.368678,65427.020625,61787.007254,53362.700777,9538.748442,1,9538.748442,0.082044,Neutral Overbought,2.364675
4952,2024-05-14,61552.789062,62900.773438,63092.125000,61123.765625,2.818627e+10,62900.773438,63092.125000,61123.765625,2.818627e+10,...,43543.831563,65258.900156,61675.289062,53387.886563,8164.902500,1,8164.902500,0.071036,Neutral Overbought,-2.144084
4953,2024-05-15,66267.492188,61553.988281,66454.453125,61330.410156,3.981517e+10,61553.988281,66454.453125,61330.410156,3.981517e+10,...,43665.416484,65184.493281,62400.939174,53457.242441,12810.249746,1,12810.249746,0.106998,Neutral Overbought,7.659609
4954,2024-05-16,65231.582031,66256.109375,66712.429688,64613.054688,3.157308e+10,66256.109375,66712.429688,64613.054688,3.157308e+10,...,43783.159277,65100.018047,62712.599330,53519.486797,11712.095234,1,11712.095234,0.098627,Neutral Overbought,-1.563225
4955,2024-05-17,67051.875000,65231.296875,67459.460938,65119.316406,2.803128e+10,65231.296875,67459.460938,65119.316406,2.803128e+10,...,43907.426185,65026.156484,63606.756138,53590.537765,13461.337235,1,13461.337235,0.111580,Neutral Overbought,2.790509
4956,2024-05-18,66940.804688,67066.210938,67387.328125,66663.500000,1.671228e+10,67066.210938,67387.328125,66663.500000,1.671228e+10,...,44030.280404,64967.116016,64484.912388,53665.597387,13275.207300,1,13275.207300,0.110071,Neutral Overbought,-0.165648
4957,2024-05-19,66278.367188,66937.929688,67694.296875,65937.179688,1.924909e+10,66937.929688,67694.296875,65937.179688,1.924909e+10,...,44153.951908,64899.777266,65174.908482,53735.776827,12542.590360,1,12542.590360,0.104509,Neutral Overbought,-0.989587
4958,2024-05-20,71448.195312,66278.742188,71483.562500,66086.171875,4.385066e+10,66278.742188,71483.562500,66086.171875,4.385066e+10,...,44294.687923,64902.068203,66395.872210,53844.302400,17603.892912,1,17603.892912,0.140502,Neutral Overbought,7.800174
4959,2024-05-21,70136.531250,71443.062500,71946.460938,69191.125000,4.693201e+10,71443.062500,71946.460938,69191.125000,4.693201e+10,...,44430.626452,64910.755859,67622.121094,53951.518140,16185.013110,1,16185.013110,0.130432,Neutral Overbought,-1.835825
4960,2024-05-23,69523.421875,69134.007812,69550.257812,69134.007812,3.256031e+10,69134.007812,69550.257812,69134.007812,3.256031e+10,...,44565.002227,64992.284844,68087.253906,54080.888437,15442.533438,1,15442.533438,0.124935,Neutral Overbought,-0.874166


In [99]:
df = df[['date', 'price', 'daily_pct_change']]
df


,date,price,daily_pct_change
0,2010-09-25,0.100000,NaN
1,2010-09-26,0.100000,0.000000
2,2010-09-27,0.100000,0.000000
3,2010-09-28,0.100000,0.000000
4,2010-09-29,0.100000,0.000000
...,...,...,...
4956,2024-05-18,66940.804688,-0.165648
4957,2024-05-19,66278.367188,-0.989587
4958,2024-05-20,71448.195312,7.800174
4959,2024-05-21,70136.531250,-1.835825


In [100]:
# Convert the dates to datetime objects
date1 = pd.to_datetime('2012-11-28')
date2 = pd.to_datetime('2016-07-9')
date3 = pd.to_datetime('2020-05-11')
date4 = pd.to_datetime('2024-04-20')

# Create the cycle DataFrames
cycle1 = df[df['date'] <= date1]
cycle2 = df[(df['date'] > date1) & (df['date'] <= date2)]
cycle3 = df[(df['date'] > date2) & (df['date'] <= date3)]
cycle4 = df[(df['date'] > date3) & (df['date'] <= date4)]
cycle5 = df[df['date'] > date4]

cycle1 = cycle1.reset_index(drop=True)
cycle2 = cycle2.reset_index(drop=True)
cycle3 = cycle3.reset_index(drop=True)
cycle4 = cycle4.reset_index(drop=True)
cycle5 = cycle5.reset_index(drop=True)


cycle1['cumulative_pct_change'] = cycle1['daily_pct_change'].cumsum()
cycle2['cumulative_pct_change'] = cycle2['daily_pct_change'].cumsum()
cycle3['cumulative_pct_change'] = cycle3['daily_pct_change'].cumsum()
cycle4['cumulative_pct_change'] = cycle4['daily_pct_change'].cumsum()
cycle5['cumulative_pct_change'] = cycle5['daily_pct_change'].cumsum()

cycle1 = cycle1.reset_index()
cycle2 = cycle2.reset_index()
cycle3 = cycle3.reset_index()
cycle4 = cycle4.reset_index()
cycle5 = cycle5.reset_index()

cycle1 = cycle1.rename(columns={'index': 'days from halving'})

cycle2 = cycle2.rename(columns={'index': 'days from halving'})
cycle3 = cycle3.rename(columns={'index': 'days from halving'})
cycle4 = cycle4.rename(columns={'index': 'days from halving'})
cycle5 = cycle5.rename(columns={'index': 'days from halving'})

cycle5



,days from halving,date,price,daily_pct_change,cumulative_pct_change
0,0,2024-04-21,64926.644531,-0.104312,-0.104312
1,1,2024-04-22,66837.679688,2.943376,2.839064
2,2,2024-04-23,66407.273438,-0.643957,2.195107
3,3,2024-04-24,64276.898438,-3.208045,-1.012938
4,4,2024-04-25,64481.707031,0.318635,-0.694303
5,5,2024-04-26,63755.320312,-1.126500,-1.820804
6,6,2024-04-27,63419.140625,-0.527297,-2.348100
7,7,2024-04-28,63113.230469,-0.482363,-2.830463
8,8,2024-04-29,63841.121094,1.153309,-1.677154
9,9,2024-04-30,60636.855469,-5.019125,-6.696279


In [101]:
# Get the first price value from cycle 5
first_price_cycle5 = cycle5['price'].iloc[0]

# cycle1['new_price'] = cycle1['price'].pct_change().add(1).fillna(1).cumprod().mul(first_price_cycle5)
cycles = [cycle1, cycle2, cycle3, cycle4]

for cycle in cycles:
    cycle['new_price'] = cycle['price'].pct_change().add(1).fillna(1).cumprod().mul(first_price_cycle5)
cycle1


,days from halving,date,price,daily_pct_change,cumulative_pct_change,new_price
0,0,2010-09-25,0.1,NaN,NaN,6.492664e+04
1,1,2010-09-26,0.1,0.000000,0.000000,6.492664e+04
2,2,2010-09-27,0.1,0.000000,0.000000,6.492664e+04
3,3,2010-09-28,0.1,0.000000,0.000000,6.492664e+04
4,4,2010-09-29,0.1,0.000000,0.000000,6.492664e+04
...,...,...,...,...,...,...
791,791,2012-11-24,12.4,0.000000,854.106991,8.050904e+06
792,792,2012-11-25,12.5,0.806452,854.913443,8.115831e+06
793,793,2012-11-26,12.3,-1.600000,853.313443,7.985977e+06
794,794,2012-11-27,12.2,-0.813008,852.500435,7.921051e+06


In [102]:
import pandas as pd
from datetime import timedelta

# Add new_date column to cycle3 DataFrame
cycle3['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle3['days from halving'], unit='D')
# cycle1['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle1['days from halving'], unit='D')
cycle2['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle2['days from halving'], unit='D')
# Add new_date column to cycle4 DataFrame
cycle4['new_date'] = pd.to_datetime('2024-04-20') + pd.to_timedelta(cycle4['days from halving'], unit='D')


In [103]:
import plotly.graph_objects as go
import pandas as pd

# Create a new figure
fig = go.Figure()

# Calculate the average of new_price for cycle3 and cycle4
# Calculate the median of new_price for cycle3, cycle2, and cycle4
average_price = pd.concat([cycle3['new_price'], cycle2['new_price'], cycle4['new_price']], axis=1).mean(axis=1)

# Convert the average_price Series into a DataFrame
average_price = average_price.to_frame()

# Rename the 0 column to 'new_price'
average_price.columns = ['new_price']

# Add a new column called 'new_date' and use the 'new_date' column from cycle4
average_price['new_date'] = cycle4['new_date']

# # Add a trace for the average_price
fig.add_trace(go.Scatter(x=average_price.new_date, y=average_price.new_price, mode='lines', name='Average'))

# Add traces for new_price for cycle3 and cycle4
fig.add_trace(go.Scatter(x=cycle2.new_date, y=cycle2['new_price'], mode='lines', name='2012-2016 Cycle'))
fig.add_trace(go.Scatter(x=cycle3.new_date, y=cycle3['new_price'], mode='lines', name='2016-2020 Cycle'))
fig.add_trace(go.Scatter(x=cycle4.new_date, y=cycle4['new_price'], mode='lines', name='2020-2024 Cycle'))
fig.add_trace(go.Scatter(x=cycle5.date, y=cycle5['price'], mode='lines', name='2024-2028 Cycle'))
# Set labels and title
fig.update_layout(
    title='Cycles 3-4',
    xaxis_title='Index',
    yaxis_title='New Price',
    width=1500,  # Set the width of the plot
    height=900  # Set the height of the plot
)



# Set y-axis to logarithmic scale
fig.update_yaxes(type='log')

# Show the plot
fig.show()

In [110]:
from datetime import date

# Get the most recent 'date' from cycle5
cycle5['date'] = pd.to_datetime(cycle5['date'])
recent_date = cycle5.iloc[-1]['date']

# Filter 'new_date' and 'new_price' for the most recent date
cycle2_today = cycle2[cycle2['new_date'] == pd.Timestamp(recent_date)][['new_date', 'new_price']]
cycle3_today = cycle3[cycle3['new_date'] == pd.Timestamp(recent_date)][['new_date', 'new_price']]
cycle4_today = cycle4[cycle4['new_date'] == pd.Timestamp(recent_date)][['new_date', 'new_price']]
cycle5_today = cycle5[cycle5['date'] == pd.Timestamp(recent_date)][['date', 'price']]

# Concatenate the dataframes and rename the 'new_price' or 'price' column
cycle2_today.rename(columns={'new_date': 'date', 'new_price': 'cycle2_price'}, inplace=True)
cycle3_today.rename(columns={'new_date': 'date', 'new_price': 'cycle3_price'}, inplace=True)
cycle4_today.rename(columns={'new_date': 'date', 'new_price': 'cycle4_price'}, inplace=True)
cycle5_today.rename(columns={'price': 'cycle5_price'}, inplace=True)

# Merge the dataframes on 'date'
compare_df = pd.merge(cycle2_today, cycle3_today, on='date', how='outer')
compare_df = pd.merge(compare_df, cycle4_today, on='date', how='outer')
compare_df = pd.merge(compare_df, cycle5_today, on='date', how='outer')

compare_df

,date,cycle2_price,cycle3_price,cycle4_price,cycle5_price
0,2024-05-23,69639.062279,58906.556953,68841.926427,69523.421875


In [113]:
import plotly.subplots as sp

# Create a 2x2 grid of subplots of type 'indicator'
fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Cycle 2 Price", "Cycle 3 Price", "Cycle 4 Price", "Cycle 5 Price"), specs=[[{'type': 'indicator'}, {'type': 'indicator'}], [{'type': 'indicator'}, {'type': 'indicator'}]])

# Add traces for each cycle
fig.add_trace(go.Indicator(
    mode = "number",
    value = compare_df['cycle2_price'].iloc[-1]),
    row=1, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = compare_df['cycle3_price'].iloc[-1]),
    row=1, col=2)

fig.add_trace(go.Indicator(
    mode = "number",
    value = compare_df['cycle4_price'].iloc[-1]),
    row=2, col=1)

fig.add_trace(go.Indicator(
    mode = "number",
    value = compare_df['cycle5_price'].iloc[-1]),
    row=2, col=2)

fig.show()

In [105]:
days_df = df[['date', 'price']].copy()
import pandas as pd

# Define the bin ranges
bins = [0.01,0.1, 1, 10, 100, 1000, 10000, 100000]

# Create the bin labels
bin_labels = ['0.01-0.1','0.1-1', '1-10', '10-100', '100-1000', '1000-10000', '10000-100000']
# Cut the 'days_df' DataFrame by the bins and create a new column 'bin_label'
days_df['bin_label'] = pd.cut(days_df['price'], bins=bins, labels=bin_labels)

In [106]:
import pandas as pd

# Assuming the average_price variable contains the DataFrame
# with the 'bin_labels' column

# Group by 'bin_labels' and count the occurrences
count_by_bin = days_df.groupby('bin_label').count()

count_by_bin

C:\Users\chris\AppData\Local\Temp\ipykernel_19028\1220810037.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



,date,price
bin_label,,
0.01-0.1,31,31
0.1-1,169,169
1-10,406,406
10-100,353,353
100-1000,1354,1354
1000-10000,1095,1095
10000-100000,1553,1553
